In [0]:
%sql
DECLARE OR REPLACE VARIABLE v_manifest_table string;
-- set var v_manifest_table = 'nishant_deshpande.tmp.manifest_11';
-- set var v_manifest_table = 'nishant_deshpande.tmp.manifest_12';
-- set var v_manifest_table = 'nishant_deshpande.tmp.manifest_test_20250611_2';
-- set var v_manifest_table = 'nishant_deshpande.tmp.manifest_nishant_deshpande__tmp__tpch_orders_20250610_1_20250612_181515'
set var v_manifest_table = 'nishant_deshpande.tmp.manifest_test_20250612_175844'

In [0]:
%sql
desc IDENTIFIER(v_manifest_table)

In [0]:
%sql
with latest_table_files as (
  select distinct table_name, file_name, file_size
  from IDENTIFIER(v_manifest_table)
  where latest_version = 1
),
retention_table_files as (
  select distinct table_name, file_name, file_size
  from IDENTIFIER(v_manifest_table)
  where in_retention_period = 1
),
all_table_files as (
  select distinct table_name, file_name, file_size
  from IDENTIFIER(v_manifest_table)
),
table_ranking as (
  select table_name, 
    sum(file_size) as total_table_file_size, 
    row_number() over (order by sum(file_size) desc) as table_rank
  from retention_table_files
  group by 1
),
table_union as (
  select 'latest' as type, table_name, 
    -- min(table_version_number) as min_tv, max(table_version_number) as max_tv, 
    count(1) as num_files, sum(file_size) as total_file_size
  from latest_table_files
  group by 1, 2
  union all
  select 'in_retention' as type, table_name, 
    -- min(table_version_number) as min_tv, max(table_version_number) as max_tv, 
    count(1) as num_files, sum(file_size) as total_file_size
  from retention_table_files
  group by 1, 2
  union all
  select 'all' as type, table_name, 
    -- min(table_version_number) as min_tv, max(table_version_number) as max_tv, 
    count(1) as num_files, sum(file_size) as total_file_size
  from all_table_files
  group by 1, 2
)
select type, table_name, num_files, total_file_size, table_rank
from table_union
join table_ranking using(table_name)
-- order by total_file_size desc, type desc
-- order by table_name desc, type desc
order by table_rank asc, type desc
;